# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 26 2022 11:39AM,249425,10,8692286,Eywa Pharma Inc.,Released
1,Oct 26 2022 11:39AM,249425,10,8692287,Eywa Pharma Inc.,Released
2,Oct 26 2022 11:39AM,249425,10,8692315,Eywa Pharma Inc.,Released
3,Oct 26 2022 11:39AM,249425,10,8691195-BO,Eywa Pharma Inc.,Released
4,Oct 26 2022 11:39AM,249425,10,8692264-BO,Eywa Pharma Inc.,Released
5,Oct 26 2022 11:39AM,249425,10,8691185-BO,Eywa Pharma Inc.,Released
6,Oct 26 2022 11:39AM,249425,10,8691194-BO,Eywa Pharma Inc.,Released
7,Oct 26 2022 11:39AM,249425,10,8692265-BO,Eywa Pharma Inc.,Released
8,Oct 26 2022 11:39AM,249425,10,8692266-BO,Eywa Pharma Inc.,Released
9,Oct 26 2022 11:39AM,249425,10,8692263-BO,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,249419,Executing,1
20,249422,Released,1
21,249424,Released,1
22,249425,Released,13
23,249426,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249419,1.0,NaN
249422,NaN,1.0
249424,NaN,1.0
249425,NaN,13.0
249426,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249344,0.0,1.0
249372,0.0,1.0
249382,0.0,1.0
249386,1.0,0.0
249388,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249344,0,1
249372,0,1
249382,0,1
249386,1,0
249388,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249344,0,1
1,249372,0,1
2,249382,0,1
3,249386,1,0
4,249388,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249344,,1
1,249372,,1
2,249382,,1
3,249386,1,
4,249388,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 26 2022 11:39AM,249425,10,Eywa Pharma Inc.
13,Oct 26 2022 11:38AM,249426,19,"GCH Granules USA, Inc."
14,Oct 26 2022 11:35AM,249424,21,"NBTY Global, Inc."
15,Oct 26 2022 11:06AM,249412,15,"Alliance Pharma, Inc."
38,Oct 26 2022 10:55AM,249422,19,"GCH Granules USA, Inc."
39,Oct 26 2022 10:40AM,249419,16,Sartorius Bioprocess Solutions
40,Oct 26 2022 10:38AM,249418,12,"SD Head USA, LLC"
41,Oct 26 2022 10:36AM,249414,15,"Mizner Bioscience, LLC"
44,Oct 26 2022 10:35AM,249413,15,"Brookfield Pharmaceuticals, LLC"
63,Oct 26 2022 10:30AM,249410,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 26 2022 11:39AM,249425,10,Eywa Pharma Inc.,,13
1,Oct 26 2022 11:38AM,249426,19,"GCH Granules USA, Inc.",,1
2,Oct 26 2022 11:35AM,249424,21,"NBTY Global, Inc.",,1
3,Oct 26 2022 11:06AM,249412,15,"Alliance Pharma, Inc.",,23
4,Oct 26 2022 10:55AM,249422,19,"GCH Granules USA, Inc.",,1
5,Oct 26 2022 10:40AM,249419,16,Sartorius Bioprocess Solutions,1,
6,Oct 26 2022 10:38AM,249418,12,"SD Head USA, LLC",,1
7,Oct 26 2022 10:36AM,249414,15,"Mizner Bioscience, LLC",2,1
8,Oct 26 2022 10:35AM,249413,15,"Brookfield Pharmaceuticals, LLC",19,
9,Oct 26 2022 10:30AM,249410,10,ISDIN Corporation,4,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 26 2022 11:39AM,249425,10,Eywa Pharma Inc.,13,
1,Oct 26 2022 11:38AM,249426,19,"GCH Granules USA, Inc.",1,
2,Oct 26 2022 11:35AM,249424,21,"NBTY Global, Inc.",1,
3,Oct 26 2022 11:06AM,249412,15,"Alliance Pharma, Inc.",23,
4,Oct 26 2022 10:55AM,249422,19,"GCH Granules USA, Inc.",1,
5,Oct 26 2022 10:40AM,249419,16,Sartorius Bioprocess Solutions,,1
6,Oct 26 2022 10:38AM,249418,12,"SD Head USA, LLC",1,
7,Oct 26 2022 10:36AM,249414,15,"Mizner Bioscience, LLC",1,2
8,Oct 26 2022 10:35AM,249413,15,"Brookfield Pharmaceuticals, LLC",,19
9,Oct 26 2022 10:30AM,249410,10,ISDIN Corporation,2,4


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 26 2022 11:39AM,249425,10,Eywa Pharma Inc.,13,
1,Oct 26 2022 11:38AM,249426,19,"GCH Granules USA, Inc.",1,
2,Oct 26 2022 11:35AM,249424,21,"NBTY Global, Inc.",1,
3,Oct 26 2022 11:06AM,249412,15,"Alliance Pharma, Inc.",23,
4,Oct 26 2022 10:55AM,249422,19,"GCH Granules USA, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 26 2022 11:39AM,249425,10,Eywa Pharma Inc.,13.0,NaN
1,Oct 26 2022 11:38AM,249426,19,"GCH Granules USA, Inc.",1.0,NaN
2,Oct 26 2022 11:35AM,249424,21,"NBTY Global, Inc.",1.0,NaN
3,Oct 26 2022 11:06AM,249412,15,"Alliance Pharma, Inc.",23.0,NaN
4,Oct 26 2022 10:55AM,249422,19,"GCH Granules USA, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 26 2022 11:39AM,249425,10,Eywa Pharma Inc.,13.0,0.0
1,Oct 26 2022 11:38AM,249426,19,"GCH Granules USA, Inc.",1.0,0.0
2,Oct 26 2022 11:35AM,249424,21,"NBTY Global, Inc.",1.0,0.0
3,Oct 26 2022 11:06AM,249412,15,"Alliance Pharma, Inc.",23.0,0.0
4,Oct 26 2022 10:55AM,249422,19,"GCH Granules USA, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1496396,49.0,4.0
12,249418,1.0,0.0
15,748239,24.0,21.0
16,249419,0.0,1.0
19,997602,4.0,0.0
21,1496384,5.0,1.0
22,249408,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1496396,49.0,4.0
1,12,249418,1.0,0.0
2,15,748239,24.0,21.0
3,16,249419,0.0,1.0
4,19,997602,4.0,0.0
5,21,1496384,5.0,1.0
6,22,249408,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,49.0,4.0
1,12,1.0,0.0
2,15,24.0,21.0
3,16,0.0,1.0
4,19,4.0,0.0
5,21,5.0,1.0
6,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,49.0
1,12,Released,1.0
2,15,Released,24.0
3,16,Released,0.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Executing,4.0,0.0,21.0,1.0,0.0,1.0,0.0
Released,49.0,1.0,24.0,0.0,4.0,5.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Executing,4.0,0.0,21.0,1.0,0.0,1.0,0.0
1,Released,49.0,1.0,24.0,0.0,4.0,5.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Executing,4.0,0.0,21.0,1.0,0.0,1.0,0.0
1,Released,49.0,1.0,24.0,0.0,4.0,5.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()